In [38]:
import numpy as np
import pandas as pd

In [39]:

pd.options.display.max_rows = 1000


train = pd.read_csv('C:/Users/1/Downloads/Telegram Desktop/train.csv')
test = pd.read_csv('C:/Users/1/Downloads/Telegram Desktop/test.csv')

train.shape, test.shape

((20483, 61), (9988, 59))

In [40]:
result = (
    train
    .isnull().sum()                   # count of non-null values
    .sort_values(ascending=False)     # sort values using descending order
)

result[result > 0]

state                          12014
build_year                     11582
num_room                        9572
max_floor                       9572
cafe_sum_1000_min_price_avg     4596
life_sq                         4193
build_count_mix                 3063
build_count_wood                3063
cafe_sum_1500_min_price_avg     2998
total_trans_amt                 2403
0_17_all                        2403
floor                            167
metro_km_walk                     10
railroad_station_walk_km          10
dtype: int64

All variables are numeric, so we can fill in missing values by mode

In [41]:
for i in result[result > 0].index:
    train_modes = train.dropna().mode().iloc[0]
    train[i].fillna(train_modes[i], inplace=True)
    test[i].fillna(train_modes[i], inplace=True)

In [42]:
result = (
    train
    .isnull().sum()                   # count of non-null values
    .sort_values(ascending=False)     # sort values using descending order
)

result[result > 0]

Series([], dtype: int64)

We will use z-score to estimate are there outliers in features

By this strategy we need to use standart scaler 

In [43]:
from sklearn.preprocessing import StandardScaler

In [44]:
tr_sc = StandardScaler()

for i in train.select_dtypes(include=['float64',"int64"]).columns:
    if i!="__churn" and i!="__price_doc":
        tr_sc.fit(np.array(train[i]).reshape(-1, 1))
        train[f'f__{i}'] = tr_sc.transform(np.array(train[i]).reshape(-1, 1))
        test[f'f__{i}'] = tr_sc.transform(np.array(test[i]).reshape(-1, 1))

In [45]:
#check correct transformation
f_list=[]
for i in train.select_dtypes(include=['float64',"int64"]).columns:
    if "f__" in i:
        f_list.append(i)
        print(f"feature: {i} mean: {train[i].mean()} std: {train[i].std()}")

feature: f__max_floor mean: 1.8006480833936114e-14 std: 1.0000244113805672
feature: f__state mean: 8.787515784931337e-15 std: 1.000024411380634
feature: f__total_revolving_bal mean: -1.617772133768835e-16 std: 1.0000244113805745
feature: f__market_count_1500 mean: 1.1105807589395894e-15 std: 1.0000244113806422
feature: f__leisure_count_3000 mean: 5.854213651354258e-16 std: 1.0000244113806045
feature: f__total_ct_chng_q4_q1 mean: 7.02866624607075e-18 std: 1.0000244113805863
feature: f__railroad_station_walk_km mean: 2.355883718672864e-16 std: 1.0000244113806036
feature: f__contacts_count_12_mon mean: -2.645174244385924e-16 std: 1.0000244113807104
feature: f__0_17_all mean: -6.688905370829802e-15 std: 1.0000244113806522
feature: f__trc_count_2000 mean: -4.832611172804489e-16 std: 1.0000244113806043
feature: f__build_count_wood mean: 2.691300290080208e-15 std: 1.0000244113806225
feature: f__credit_limit mean: -5.978499224535213e-18 std: 1.0000244113805914
feature: f__total_trans_ct mean: 

Delete all observations where (at least) one of (standart) random variable more that 3 (it means that this variable deviate more that 3 standart deviation from mean)

In [46]:
train_without_out = train[(np.abs(train[f_list])<3).all(axis=1)].reset_index().drop(["index"],axis=1)
#test_without_out = test[(np.abs(test[f_list])<3).all(axis=1)].reset_index().drop(["index"],axis=1)
test_without_out = test.reset_index().drop(["index"],axis=1)

At the end we need to transform categorical variables

In [47]:
from sklearn.preprocessing import OneHotEncoder

In [48]:
onehot = OneHotEncoder(sparse=False)

for i in train.select_dtypes(include=["object"]).columns:
    if i!= "timestamp":
        onehot.fit(test_without_out[[i]])
        values_train = onehot.transform(train_without_out[[i]])
        values_test = onehot.transform(test_without_out[[i]])

        list_names = []
        for j, cat in enumerate(onehot.categories_[0]):
            name = f'f__{i}_{cat}'
            list_names.append(name)
            train_without_out[name] = values_train[:, j]
            test_without_out[name] = values_test[:, j]

In [49]:
train_without_out.to_csv('train_corr.csv', index=False)
test_without_out.to_csv('test_corr.csv', index=False)

In [50]:
train_without_out.shape, test_without_out.shape

((13953, 147), (9988, 145))

In [51]:
new_columns = train_without_out.columns[train_without_out.columns.str.startswith('f__')]
print(len(new_columns))
print(new_columns)

86
Index(['f__max_floor', 'f__state', 'f__total_revolving_bal',
       'f__market_count_1500', 'f__leisure_count_3000',
       'f__total_ct_chng_q4_q1', 'f__railroad_station_walk_km',
       'f__contacts_count_12_mon', 'f__0_17_all', 'f__trc_count_2000',
       'f__build_count_wood', 'f__credit_limit', 'f__total_trans_ct',
       'f__leisure_count_5000', 'f__life_sq', 'f__cafe_count_1000_price_1000',
       'f__mkad_km', 'f__school_education_centers_top_20_raion',
       'f__avg_utilization_ratio', 'f__public_transport_station_min_walk',
       'f__customer_age', 'f__detention_facility_km', 'f__sport_count_2000',
       'f__cafe_sum_1000_min_price_avg', 'f__total_amt_chng_q4_q1',
       'f__metro_km_walk', 'f__office_sqm_5000', 'f__total_trans_amt',
       'f__months_inactive_12_mon', 'f__cafe_sum_1500_min_price_avg',
       'f__floor', 'f__num_room', 'f__months_on_book', 'f__dependent_count',
       'f__avg_open_to_buy', 'f__build_year', 'f__full_sq',
       'f__total_relationship_cou